# Iter 1: Data overview & preparation

In this part we will overview what data contains these datasets. 

We will check data quality wether duplicates or missing values appears.

Besides we will apply standartisation and normalisation if the data needs that.

If possible we will aggregate datasets to joined tables to have certain objects and their attributes in one dataset

In [30]:
import sqlite3
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, clear_output
import os
import itertools
from datetime import datetime

from generic_functions import check_duplicates, check_missing_values

In [2]:
os.getcwd()

'/Users/kristinazekiene/Documents/LearningPython/TuringCollege/M2Capstone'

## Data loading

Let's load the database and check what tables SQLite file has inside:

In [3]:
input_path = "/Users/kristinazekiene/Documents/LearningPython/TuringCollege/M2Capstone/Input/database.sqlite"
con = sqlite3.connect(input_path)

table = pd.read_sql('''
            SELECT * FROM sqlite_master 
            WHERE TYPE = 'table'; ''',con)
table

,type,name,tbl_name,rootpage,sql
0,table,sqlite_sequence,sqlite_sequence,4,"CREATE TABLE sqlite_sequence(name,seq)"
1,table,Player_Attributes,Player_Attributes,11,"CREATE TABLE ""Player_Attributes"" (\n\t`id`\tIN..."
2,table,Player,Player,14,CREATE TABLE `Player` (\n\t`id`\tINTEGER PRIMA...
3,table,Match,Match,18,CREATE TABLE `Match` (\n\t`id`\tINTEGER PRIMAR...
4,table,League,League,24,CREATE TABLE `League` (\n\t`id`\tINTEGER PRIMA...
5,table,Country,Country,26,CREATE TABLE `Country` (\n\t`id`\tINTEGER PRIM...
6,table,Team,Team,29,"CREATE TABLE ""Team"" (\n\t`id`\tINTEGER PRIMARY..."
7,table,Team_Attributes,Team_Attributes,2,CREATE TABLE `Team_Attributes` (\n\t`id`\tINTE...


We will check what columns and data types has each table

In [4]:
for row in table["sql"]:
    print(row)

CREATE TABLE sqlite_sequence(name,seq)
CREATE TABLE "Player_Attributes" (
	`id`	INTEGER PRIMARY KEY AUTOINCREMENT,
	`player_fifa_api_id`	INTEGER,
	`player_api_id`	INTEGER,
	`date`	TEXT,
	`overall_rating`	INTEGER,
	`potential`	INTEGER,
	`preferred_foot`	TEXT,
	`attacking_work_rate`	TEXT,
	`defensive_work_rate`	TEXT,
	`crossing`	INTEGER,
	`finishing`	INTEGER,
	`heading_accuracy`	INTEGER,
	`short_passing`	INTEGER,
	`volleys`	INTEGER,
	`dribbling`	INTEGER,
	`curve`	INTEGER,
	`free_kick_accuracy`	INTEGER,
	`long_passing`	INTEGER,
	`ball_control`	INTEGER,
	`acceleration`	INTEGER,
	`sprint_speed`	INTEGER,
	`agility`	INTEGER,
	`reactions`	INTEGER,
	`balance`	INTEGER,
	`shot_power`	INTEGER,
	`jumping`	INTEGER,
	`stamina`	INTEGER,
	`strength`	INTEGER,
	`long_shots`	INTEGER,
	`aggression`	INTEGER,
	`interceptions`	INTEGER,
	`positioning`	INTEGER,
	`vision`	INTEGER,
	`penalties`	INTEGER,
	`marking`	INTEGER,
	`standing_tackle`	INTEGER,
	`sliding_tackle`	INTEGER,
	`gk_diving`	INTEGER,
	`gk_handling`

## Datasets overview

### Player Attributes

In [35]:
pa_df = pd.read_sql_query("SELECT * from Player_Attributes", con)
print("Player Attributes table shape:",pa_df.shape)

check_duplicates(pa_df)
check_missing_values(pa_df)

display(pa_df.head(4))

Player Attributes table shape: (183978, 42)
Table does not have duplicate rows
Table has missing values


,id,player_fifa_api_id,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,1,218353,505942,2016-02-18 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
1,2,218353,505942,2015-11-19 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
2,3,218353,505942,2015-09-21 00:00:00,62.0,66.0,right,medium,medium,49.0,...,54.0,48.0,65.0,66.0,69.0,6.0,11.0,10.0,8.0,8.0
3,4,218353,505942,2015-03-20 00:00:00,61.0,65.0,right,medium,medium,48.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0


In [36]:
pa_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 183978 entries, 0 to 183977
Data columns (total 42 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   id                   183978 non-null  int64  
 1   player_fifa_api_id   183978 non-null  int64  
 2   player_api_id        183978 non-null  int64  
 3   date                 183978 non-null  object 
 4   overall_rating       183142 non-null  float64
 5   potential            183142 non-null  float64
 6   preferred_foot       183142 non-null  object 
 7   attacking_work_rate  180748 non-null  object 
 8   defensive_work_rate  183142 non-null  object 
 9   crossing             183142 non-null  float64
 10  finishing            183142 non-null  float64
 11  heading_accuracy     183142 non-null  float64
 12  short_passing        183142 non-null  float64
 13  volleys              181265 non-null  float64
 14  dribbling            183142 non-null  float64
 15  curve            

We can see that table has missing values. Lets remove them:

In [55]:
pa_dfm = pa_df.dropna()
print('Tables lenght after the missing values removal:')
print(pa_dfm.shape)

pa_dfm.describe(include='all').T

Tables lenght after the missing values removal:
(180354, 42)


,count,unique,top,freq,mean,std,min,25%,50%,75%,max
id,180354.0,NaN,NaN,NaN,91995.886274,53092.657914,1.0,46074.25,92003.5,137935.75,183978.0
player_fifa_api_id,180354.0,NaN,NaN,NaN,166822.125803,52821.443279,2.0,156616.0,183792.0,200138.0,234141.0
player_api_id,180354.0,NaN,NaN,NaN,137653.145514,137599.735284,2625.0,35451.0,80291.0,192841.0,750584.0
date,180354,197,2007-02-22 00:00:00,10410,NaN,NaN,NaN,NaN,NaN,NaN,NaN
overall_rating,180354.0,NaN,NaN,NaN,68.635317,7.02795,33.0,64.0,69.0,73.0,94.0
potential,180354.0,NaN,NaN,NaN,73.479457,6.581963,39.0,69.0,74.0,78.0,97.0
preferred_foot,180354,2,right,136247,NaN,NaN,NaN,NaN,NaN,NaN,NaN
attacking_work_rate,180354,8,medium,125070,NaN,NaN,NaN,NaN,NaN,NaN,NaN
defensive_work_rate,180354,18,medium,130846,NaN,NaN,NaN,NaN,NaN,NaN,NaN
crossing,180354.0,NaN,NaN,NaN,55.142071,17.247231,1.0,45.0,59.0,68.0,95.0


All players attributes from `crossing`, to `gk_reflexes` is on the same scale up to 100.
The median and mean statistics shows that their values are in similar range (around 40-60). Few attributes statistics are lower.

In [56]:
print('# of unique players in the dataset:')
pa_dfm.player_api_id.nunique()

# of unique players in the dataset:


10410

In [57]:
foot = (pa_dfm.groupby('preferred_foot')
            .agg(Occur=('player_api_id','nunique'))
            .reset_index()
       )
foot['Share'] = round(foot.Occur / foot.Occur.sum() * 100,1)
foot

,preferred_foot,Occur,Share
0,left,3048,26.5
1,right,8456,73.5


Majority of players preffers right foot (73.5%). 

However, there are still close to 1/3 of players who prefers left foot (26.5%)

<b>Work Rate</b>

Known as Player Work Rate. Player Work Rate is the rate of a player's behavior on the pitch in terms of attacking and defensive works. The Work Rate is defined for <b>Attacking (Attacking Work Rate - Abbreviated as AWR) and Defensive (Defensive Work Rate - Abbreviated as DWR)</b> and rated between low, medium and high, which defines how a player puts effort to participates in attacks and defenses even when they are out of position.

A Player's Work Rate is shown in their Bio page.

- <b>Attacking Work Rate (AWR)</b> -
How a player participates in attacks. Rated as low, medium and high.

- <b>Defensive Work Rate (DWR)</b> -
How a player participates in defensive plays. Rated as low, medium and high.

In [58]:
AWR = (pa_dfm.groupby('attacking_work_rate')
            .agg(Occur=('player_api_id','nunique'))
            .reset_index()
       )
AWR['Share'] = round(AWR.Occur 
                 / AWR.Occur.sum() * 100,1)
AWR

,attacking_work_rate,Occur,Share
0,None,407,3.0
1,high,3360,25.2
2,le,14,0.1
3,low,851,6.4
4,medium,8642,64.7
5,norm,56,0.4
6,stoc,14,0.1
7,y,13,0.1


There are messy values on AWR attribute. We need to standartize according FIFA definitions. We will do corrections.

In [59]:
AWR_mapping = {
    'None': 'None',
    'high': 'high',
    'le': 'low',
    'low': 'low',
    'medium': 'medium',
    'norm': 'medium',
    'stoc': 'medium',
    'y': 'None'
    }

pa_dfm['AWR'] = pa_dfm.attacking_work_rate.apply(lambda x: AWR_mapping[x])
clear_output()

pa_dfm.groupby('AWR').player_api_id.count()

AWR
None        3411
high       42823
low         8647
medium    125473
Name: player_api_id, dtype: int64

In [60]:
DWR = (pa_dfm.groupby('defensive_work_rate')
            .agg(Occur=('player_api_id','nunique'))
            .reset_index()
       )
DWR['Share'] = round(DWR.Occur 
                 / DWR.Occur.sum() * 100,1)
DWR

,defensive_work_rate,Occur,Share
0,0,15,0.1
1,1,34,0.3
2,2,32,0.2
3,3,24,0.2
4,4,13,0.1
5,5,25,0.2
6,6,15,0.1
7,7,19,0.1
8,8,6,0.0
9,9,13,0.1


In [61]:
DWR_mapping = {
    '0': 'None', '1': 'None', '2': 'None',
    '3': 'None', '4': 'None', '5': 'None',
    '6': 'None', '7': 'None', '8': 'None',
    '9': 'None',
    'ean': 'None',
    'es': 'None',
    'high': 'high',
    'low': 'low',
    'medium': 'medium',
    'le': 'low',
    'o': 'None',
    'ormal': 'medium',
    'tocky': 'None'
    }

pa_dfm['DWR'] = pa_dfm.defensive_work_rate.apply(lambda x: DWR_mapping[x])
clear_output()

pa_dfm.groupby('DWR').player_api_id.count()

DWR
None        3718
high       27041
low        18432
medium    131163
Name: player_api_id, dtype: int64

In [62]:
pa_dfm['date'] = pa_dfm.date.apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S').date())
clear_output()
pa_dfm['date']

0         2016-02-18
1         2015-11-19
2         2015-09-21
3         2015-03-20
4         2007-02-22
             ...    
183973    2009-08-30
183974    2009-02-22
183975    2008-08-30
183976    2007-08-30
183977    2007-02-22
Name: date, Length: 180354, dtype: object

### Player

In [50]:
p_df = pd.read_sql_query("SELECT * from Player", con)
print("Player table shape:",p_df.shape)

check_duplicates(p_df)
check_missing_values(p_df)

display(p_df.head(4))

Player table shape: (11060, 7)
Table does not have duplicate rows
Table does not have missing values


,id,player_api_id,player_name,player_fifa_api_id,birthday,height,weight
0,1,505942,Aaron Appindangoye,218353,1992-02-29 00:00:00,182.88,187
1,2,155782,Aaron Cresswell,189615,1989-12-15 00:00:00,170.18,146
2,3,162549,Aaron Doran,186170,1991-05-13 00:00:00,170.18,163
3,4,30572,Aaron Galindo,140161,1982-05-08 00:00:00,182.88,198


In [51]:
print('# of unique players:')
p_df.player_api_id.nunique()

# of unique players:


11060

In [52]:
print('# of unique players names:')
p_df.player_name.nunique()

# of unique players names:


10848

Number of unique players identifiers count and players names count differs. Could same players have different ids and there are duplicates in the dataset?

In [53]:
same_names = (p_df.groupby('player_name').id.count()
                  .sort_values(ascending = False)
                  .head(25)
                  .reset_index())
same_names

,player_name,id
0,Danilo,7
1,Paulinho,6
2,Ricardo,5
3,Douglas,4
4,Adriano,4
5,Roberto,4
6,Joao Pedro,4
7,Fernando,4
8,Rafael,4
9,Felipe,4


In [54]:
(p_df[p_df.player_name.isin(list(same_names.player_name))]
     .sort_values(by='player_name'))

,id,player_api_id,player_name,player_fifa_api_id,birthday,height,weight
190,191,121923,Adriano,183243,1982-01-21 00:00:00,175.26,168
191,192,30690,Adriano,106019,1982-02-17 00:00:00,187.96,194
192,193,33992,Adriano,164082,1984-10-26 00:00:00,172.72,161
193,194,41109,Adriano,152769,1980-01-29 00:00:00,182.88,176
437,438,150396,Alex,186613,1989-01-29 00:00:00,182.88,170
...,...,...,...,...,...,...,...
9516,9530,25382,Scott Brown,134820,1985-06-25 00:00:00,175.26,163
9518,9532,47559,Scott Brown,152456,1985-04-26 00:00:00,187.96,192
10713,10729,267369,Wallace,204254,1994-05-01 00:00:00,175.26,161
10712,10728,109332,Wallace,187027,1986-10-29 00:00:00,182.88,165


Some players have same name, but from their attributes (like bithday, height) we can say tthey are different persons

In [64]:
p_df['birthday'] = p_df.birthday.apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S').date())
clear_output()
p_df['birthday']

0        1992-02-29
1        1989-12-15
2        1991-05-13
3        1982-05-08
4        1979-11-08
            ...    
11055    1979-04-03
11056    1986-12-18
11057    1979-04-29
11058    1981-10-06
11059    1982-06-05
Name: birthday, Length: 11060, dtype: object

In [68]:
players = pd.merge(pa_dfm, p_df, 
                how='left',
                on='player_api_id')

players.shape

(180354, 50)

In [69]:
output_p_csv = "/Users/kristinazekiene/Documents/LearningPython/TuringCollege/M2Capstone/Output/players.csv"
players.to_csv(output_p_csv, index=False)

### Match

In [71]:
matches_df = pd.read_sql_query("SELECT * from Match", con)
print("Matches table shape:",matches_df.shape)

check_duplicates(matches_df)
check_missing_values(matches_df)

display(matches_df.head(4))

Matches table shape: (25979, 115)
Table does not have duplicate rows
Table has missing values


,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
0,1,1,1,2008/2009,1,2008-08-17 00:00:00,492473,9987,9993,1,...,4.0,1.65,3.40,4.50,1.78,3.25,4.00,1.73,3.40,4.20
1,2,1,1,2008/2009,1,2008-08-16 00:00:00,492474,10000,9994,0,...,3.8,2.00,3.25,3.25,1.85,3.25,3.75,1.91,3.25,3.60
2,3,1,1,2008/2009,1,2008-08-16 00:00:00,492475,9984,8635,0,...,2.5,2.35,3.25,2.65,2.50,3.20,2.50,2.30,3.20,2.75
3,4,1,1,2008/2009,1,2008-08-17 00:00:00,492476,9991,9998,5,...,7.5,1.45,3.75,6.50,1.50,3.75,5.50,1.44,3.75,6.50


In [76]:
print(list(matches_df.columns))

['id', 'country_id', 'league_id', 'season', 'stage', 'date', 'match_api_id', 'home_team_api_id', 'away_team_api_id', 'home_team_goal', 'away_team_goal', 'home_player_X1', 'home_player_X2', 'home_player_X3', 'home_player_X4', 'home_player_X5', 'home_player_X6', 'home_player_X7', 'home_player_X8', 'home_player_X9', 'home_player_X10', 'home_player_X11', 'away_player_X1', 'away_player_X2', 'away_player_X3', 'away_player_X4', 'away_player_X5', 'away_player_X6', 'away_player_X7', 'away_player_X8', 'away_player_X9', 'away_player_X10', 'away_player_X11', 'home_player_Y1', 'home_player_Y2', 'home_player_Y3', 'home_player_Y4', 'home_player_Y5', 'home_player_Y6', 'home_player_Y7', 'home_player_Y8', 'home_player_Y9', 'home_player_Y10', 'home_player_Y11', 'away_player_Y1', 'away_player_Y2', 'away_player_Y3', 'away_player_Y4', 'away_player_Y5', 'away_player_Y6', 'away_player_Y7', 'away_player_Y8', 'away_player_Y9', 'away_player_Y10', 'away_player_Y11', 'home_player_1', 'home_player_2', 'home_player_

In [78]:
matches_df[['id', 'country_id', 'league_id', 'season', 'stage', 'date',
            'match_api_id', 'home_team_api_id', 'away_team_api_id',
            'home_team_goal', 'away_team_goal','goal', 'shoton', 'shotoff',
            'foulcommit', 'card', 'cross', 'corner', 'possession', 
            'B365H', 'B365D', 'B365A', 'BWH', 'BWD', 'BWA', 'IWH', 
            'IWD', 'IWA', 'LBH', 'LBD', 'LBA', 'PSH', 'PSD', 'PSA', 
            'WHH', 'WHD', 'WHA', 'SJH', 'SJD', 'SJA', 'VCH', 
            'VCD', 'VCA', 'GBH', 'GBD', 'GBA', 'BSH',
            'BSD', 'BSA']].describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
id,25979.0,NaN,NaN,NaN,12990.0,7499.635658,1.0,6495.5,12990.0,19484.5,25979.0
country_id,25979.0,NaN,NaN,NaN,11738.630317,7553.936759,1.0,4769.0,10257.0,17642.0,24558.0
league_id,25979.0,NaN,NaN,NaN,11738.630317,7553.936759,1.0,4769.0,10257.0,17642.0,24558.0
season,25979,8,2008/2009,3326,NaN,NaN,NaN,NaN,NaN,NaN,NaN
stage,25979.0,NaN,NaN,NaN,18.242773,10.407354,1.0,9.0,18.0,27.0,38.0
date,25979,1694,2009-04-11,60,NaN,NaN,NaN,NaN,NaN,NaN,NaN
match_api_id,25979.0,NaN,NaN,NaN,1195429.159975,494627.856527,483129.0,768436.5,1147511.0,1709852.5,2216672.0
home_team_api_id,25979.0,NaN,NaN,NaN,9984.371993,14087.453758,1601.0,8475.0,8697.0,9925.0,274581.0
away_team_api_id,25979.0,NaN,NaN,NaN,9984.475115,14087.445135,1601.0,8475.0,8697.0,9925.0,274581.0
home_team_goal,25979.0,NaN,NaN,NaN,1.544594,1.297158,0.0,1.0,1.0,2.0,10.0


In [77]:
matches_df['date'] = matches_df.date.apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S').date())
clear_output()
matches_df['date'].head(4)

0    2008-08-17
1    2008-08-16
2    2008-08-16
3    2008-08-17
Name: date, dtype: object

In [84]:
list(matches_df.possession.unique())[0:3]

[None,
 '<possession><value><comment>56</comment><event_incident_typefk>352</event_incident_typefk><elapsed>25</elapsed><subtype>possession</subtype><sortorder>1</sortorder><awaypos>44</awaypos><homepos>56</homepos><n>68</n><type>special</type><id>379029</id></value><value><comment>54</comment><elapsed_plus>1</elapsed_plus><event_incident_typefk>352</event_incident_typefk><elapsed>45</elapsed><subtype>possession</subtype><sortorder>4</sortorder><awaypos>46</awaypos><homepos>54</homepos><n>117</n><type>special</type><id>379251</id></value><value><comment>54</comment><event_incident_typefk>352</event_incident_typefk><elapsed>70</elapsed><subtype>possession</subtype><sortorder>0</sortorder><awaypos>46</awaypos><homepos>54</homepos><n>190</n><type>special</type><id>379443</id></value><value><comment>55</comment><elapsed_plus>5</elapsed_plus><event_incident_typefk>352</event_incident_typefk><elapsed>90</elapsed><subtype>possession</subtype><sortorder>1</sortorder><awaypos>45</awaypos><homep

In [87]:
print(matches_df.shape)
print('# of unique matches:',matches_df.match_api_id.nunique())

(25979, 115)
# of unique matches: 25979


In [90]:
print(matches_df.home_player_X1.unique())

[nan  1.  2.  0.]


### League

In [94]:
league_df = pd.read_sql_query("SELECT * from League", con)
print("League table shape:",league_df.shape)

check_duplicates(league_df)
check_missing_values(league_df)

display(league_df.head(4))

League table shape: (11, 3)
Table does not have duplicate rows
Table does not have missing values


,id,country_id,name
0,1,1,Belgium Jupiler League
1,1729,1729,England Premier League
2,4769,4769,France Ligue 1
3,7809,7809,Germany 1. Bundesliga


In [95]:
league_df.name.nunique()

11

In [101]:
league_df.name.unique

<bound method Series.unique of 0       Belgium Jupiler League
1       England Premier League
2               France Ligue 1
3        Germany 1. Bundesliga
4                Italy Serie A
5       Netherlands Eredivisie
6           Poland Ekstraklasa
7     Portugal Liga ZON Sagres
8      Scotland Premier League
9              Spain LIGA BBVA
10    Switzerland Super League
Name: name, dtype: object>

In [119]:
league_df = league_df.rename(columns={'name':'league',
                                     'id':'league_id'})

### Country

In [97]:
country_df = pd.read_sql_query("SELECT * from Country", con)
print("Country table shape:",country_df.shape)

check_duplicates(country_df)
check_missing_values(country_df)

display(country_df.head(4))

Country table shape: (11, 2)
Table does not have duplicate rows
Table does not have missing values


,id,name
0,1,Belgium
1,1729,England
2,4769,France
3,7809,Germany


In [98]:
country_df.name.unique

<bound method Series.unique of 0         Belgium
1         England
2          France
3         Germany
4           Italy
5     Netherlands
6          Poland
7        Portugal
8        Scotland
9           Spain
10    Switzerland
Name: name, dtype: object>

In [117]:
country_df = country_df.rename(columns={'name':'country',
                                       'id':'country_id'})
country_df

,country_id,country
0,1,Belgium
1,1729,England
2,4769,France
3,7809,Germany
4,10257,Italy
5,13274,Netherlands
6,15722,Poland
7,17642,Portugal
8,19694,Scotland
9,21518,Spain


### Team

In [103]:
team_df = pd.read_sql_query("SELECT * from Team", con)
print("Team table shape:",team_df.shape)

check_duplicates(team_df)
check_missing_values(team_df)

display(team_df.head(4))

Team table shape: (299, 5)
Table does not have duplicate rows
Table has missing values


,id,team_api_id,team_fifa_api_id,team_long_name,team_short_name
0,1,9987,673.0,KRC Genk,GEN
1,2,9993,675.0,Beerschot AC,BAC
2,3,10000,15005.0,SV Zulte-Waregem,ZUL
3,4,9994,2007.0,Sporting Lokeren,LOK


In [109]:
team_dfm = team_df.dropna()
team_dfm.shape

(288, 5)

### Team Attributes

In [111]:
ta_df = pd.read_sql_query("SELECT * from Team_Attributes", con)
print("Team Attributes table shape:",ta_df.shape)

check_duplicates(ta_df)
check_missing_values(ta_df)

display(ta_df.head(4))

Team Attributes table shape: (1458, 25)
Table does not have duplicate rows
Table has missing values


,id,team_fifa_api_id,team_api_id,date,buildUpPlaySpeed,buildUpPlaySpeedClass,buildUpPlayDribbling,buildUpPlayDribblingClass,buildUpPlayPassing,buildUpPlayPassingClass,...,chanceCreationShooting,chanceCreationShootingClass,chanceCreationPositioningClass,defencePressure,defencePressureClass,defenceAggression,defenceAggressionClass,defenceTeamWidth,defenceTeamWidthClass,defenceDefenderLineClass
0,1,434,9930,2010-02-22 00:00:00,60,Balanced,NaN,Little,50,Mixed,...,55,Normal,Organised,50,Medium,55,Press,45,Normal,Cover
1,2,434,9930,2014-09-19 00:00:00,52,Balanced,48.0,Normal,56,Mixed,...,64,Normal,Organised,47,Medium,44,Press,54,Normal,Cover
2,3,434,9930,2015-09-10 00:00:00,47,Balanced,41.0,Normal,54,Mixed,...,64,Normal,Organised,47,Medium,44,Press,54,Normal,Cover
3,4,77,8485,2010-02-22 00:00:00,70,Fast,NaN,Little,70,Long,...,70,Lots,Organised,60,Medium,70,Double,70,Wide,Cover


In [113]:
t_df = pd.merge(ta_df,team_df,
                how='left',
                on=['team_api_id','team_fifa_api_id']
               )

In [114]:
output_t_csv = "/Users/kristinazekiene/Documents/LearningPython/TuringCollege/M2Capstone/Output/teams.csv"
t_df.to_csv(output_t_csv, index=False)

In [115]:
matches_df.head(4)

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
0,1,1,1,2008/2009,1,2008-08-17,492473,9987,9993,1,...,4.0,1.65,3.40,4.50,1.78,3.25,4.00,1.73,3.40,4.20
1,2,1,1,2008/2009,1,2008-08-16,492474,10000,9994,0,...,3.8,2.00,3.25,3.25,1.85,3.25,3.75,1.91,3.25,3.60
2,3,1,1,2008/2009,1,2008-08-16,492475,9984,8635,0,...,2.5,2.35,3.25,2.65,2.50,3.20,2.50,2.30,3.20,2.75
3,4,1,1,2008/2009,1,2008-08-17,492476,9991,9998,5,...,7.5,1.45,3.75,6.50,1.50,3.75,5.50,1.44,3.75,6.50


In [126]:
matches_info_stg = pd.merge(matches_df,country_df,
                            how='left',
                            on='country_id')
matches_info_stg2 = pd.merge(matches_info_stg,league_df,
                            how='left',
                            on=['country_id','country_id'])
matches_info_stg3 = pd.merge(matches_info_stg2,team_df,
                            how='left',
                            left_on='home_team_api_id',
                            right_on='team_api_id')
matches_info_stg3 = matches_info_stg3.rename(columns={'team_long_name':'home_team_long_name',
                                                     'team_short_name':'home_team_short_name'})
matches_info_stg4 = pd.merge(matches_info_stg3,team_df,
                            how='left',
                            left_on='away_team_api_id',
                            right_on='team_api_id')
matches_info_stg4 = matches_info_stg4.rename(columns={'team_long_name':'away_team_long_name',
                                                     'team_short_name':'away_team_short_name'})
matches_info_stg4 = matches_info_stg4.drop(columns=['league_id_y', 'id_y', 'team_api_id_x', 'team_fifa_api_id_x', 
                                                    'id', 'team_api_id_y', 'team_fifa_api_id_y'])
print(list(matches_info_stg4.columns))

['id_x', 'country_id', 'league_id_x', 'season', 'stage', 'date', 'match_api_id', 'home_team_api_id', 'away_team_api_id', 'home_team_goal', 'away_team_goal', 'home_player_X1', 'home_player_X2', 'home_player_X3', 'home_player_X4', 'home_player_X5', 'home_player_X6', 'home_player_X7', 'home_player_X8', 'home_player_X9', 'home_player_X10', 'home_player_X11', 'away_player_X1', 'away_player_X2', 'away_player_X3', 'away_player_X4', 'away_player_X5', 'away_player_X6', 'away_player_X7', 'away_player_X8', 'away_player_X9', 'away_player_X10', 'away_player_X11', 'home_player_Y1', 'home_player_Y2', 'home_player_Y3', 'home_player_Y4', 'home_player_Y5', 'home_player_Y6', 'home_player_Y7', 'home_player_Y8', 'home_player_Y9', 'home_player_Y10', 'home_player_Y11', 'away_player_Y1', 'away_player_Y2', 'away_player_Y3', 'away_player_Y4', 'away_player_Y5', 'away_player_Y6', 'away_player_Y7', 'away_player_Y8', 'away_player_Y9', 'away_player_Y10', 'away_player_Y11', 'home_player_1', 'home_player_2', 'home_pla

In [127]:
matches_info_stg4.shape

(25979, 121)

In [128]:
output_m_csv = "/Users/kristinazekiene/Documents/LearningPython/TuringCollege/M2Capstone/Output/matches.csv"
matches_info_stg4.to_csv(output_m_csv, index=False)